In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
workspace,default


In [0]:
spark.sql("SHOW CATALOGS")


DataFrame[catalog: string]

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS events_db")


DataFrame[]

In [0]:
from pyspark.sql.functions import to_timestamp

# 1️⃣ Get current Unity Catalog dynamically
catalog = spark.sql("SELECT current_catalog()").collect()[0][0]

# 2️⃣ Create schema safely
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.events_db")

# 3️⃣ Sample data
data = [
    (1, "U101", "login", "2025-01-01 10:00:00", "web"),
    (2, "U101", "page_view", "2025-01-01 10:01:00", "web"),
    (3, "U101", "purchase", "2025-01-01 10:10:00", "web")
]

columns = [
    "event_id",
    "user_id",
    "event_name",
    "event_timestamp",
    "source"
]

# 4️⃣ Create DataFrame
events = spark.createDataFrame(data, columns)

# 5️⃣ Convert timestamp
events = events.withColumn(
    "event_timestamp",
    to_timestamp("event_timestamp", "yyyy-MM-dd HH:mm:ss")
)

# 6️⃣ Write Delta table in Unity Catalog
events.writeTo(f"{catalog}.events_db.events") \
    .using("delta") \
    .createOrReplace()


In [0]:
%sql
SELECT * FROM events_db.events;


event_id,user_id,event_name,event_timestamp,source
1,U101,login,2025-01-01T10:00:00.000Z,web
2,U101,page_view,2025-01-01T10:01:00.000Z,web
3,U101,purchase,2025-01-01T10:10:00.000Z,web
